In [1]:
import sys
sys.executable

'C:\\Users\\DELL\\Downloads\\ItineraireVacances\\.venv\\Scripts\\python.exe'

# Dataset — principaux-sites-touristiques-en-ile-de-france
(144 ko, 1 file, 8 columns, 1 185 downloads, Updated 10 years ago)


https://data.iledefrance.fr/explore/dataset/principaux-sites-touristiques-en-ile-de-france0/api/

# 1. Charger les données

In [14]:
import pandas as pd
import numpy as np
import pyarrow
import json
from IPython.display import display

In [16]:
#lire CSV_PATH
CSV_PATH = "principaux-sites-touristiques-en-ile-de-france.csv"
df_sites = pd.read_csv(CSV_PATH, sep=';', encoding='utf-8', low_memory = False)

#Convertir en Parquet --> Fichier beaucoup plus léger --> Lecture rapide
PARQUET_PATH = "principaux-sites-touristiques-en-ile-de-france.parquet"
df_sites.to_parquet(PARQUET_PATH, compression = "snappy")

#Libère la mémoire et recharger le parquet
del df_sites
df_sites = pd.read_parquet("principaux-sites-touristiques-en-ile-de-france.parquet")

#Afficher 2 lignes et TOUTES les colonnes
pd.set_option("display.max_columns", None)
display(df_sites.head(5))

print("\n-------------------------------------")
print("Nombre de lignes et de colones")
print(df_sites.shape)

print("\n-------------------------------------")
print("Display info sur les colonnes et datatype")
display(df_sites.info())

,Geo Point,Geo Shape,dep,adresse,nomcom,insee,nom_carto,typo_niv3,objectid
0,"48.8675639747, 2.78603920275","{""coordinates"": [2.786039202752757, 48.8675639...",77,77777 MARNE-LA-VALLÉE CEDEX 4,MARNE LA VALLÉE,77535,Business solutions - Disneyland® Paris,Centre de congrès et parc d'exposition,7
1,"48.8229494567, 2.26979827721","{""coordinates"": [2.269798277205975, 48.8229494...",92,25 AVENUE VICTOR CRESSON 92130 ISSY LES MOULIN...,ISSY LES MOULINEAUX,92040,LE PALAIS DES CONGRÈS D’ISSY,Centre de congrès et parc d'exposition,12
2,"48.8719414131, 2.21752817734","{""coordinates"": [2.217528177341401, 48.8719414...",92,123 BD WASHINGTON,SURESNES,92073,Cimetiere Americain,Site et monument,55
3,"48.4265659583, 2.16097168091","{""coordinates"": [2.160971680905903, 48.4265659...",91,"5, AVENUE CHARLES-DE-GAULLE",ETAMPES,91223,Ile de loisirs d'Etampes,"Espace de loisirs, espace récréatif",393
4,"49.0309724221, 2.04739741079","{""coordinates"": [2.047397410785327, 49.0309724...",95,RUE DES ETANGS,CERGY-PONTOISE,95127,Ile de loisirs de Cergy Pontoise,"Espace de loisirs, espace récréatif",404



-------------------------------------
Nombre de lignes et de colones
(672, 9)

-------------------------------------
Display info sur les colonnes et datatype
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Geo Point  672 non-null    object
 1   Geo Shape  672 non-null    object
 2   dep        672 non-null    int64 
 3   adresse    641 non-null    object
 4   nomcom     672 non-null    object
 5   insee      672 non-null    int64 
 6   nom_carto  568 non-null    object
 7   typo_niv3  672 non-null    object
 8   objectid   672 non-null    int64 
dtypes: int64(3), object(6)
memory usage: 47.4+ KB


None

# 2. Parser les données de la colonne "Geo Shape"

In [17]:
#parser la colonne Geo Shape en plusieurs colonnes

#Fonction de parsing
def parse_geo_shape(val):
    if pd.isna(val):
        return pd.Series([None, None, None, None])
    try:
        obj = json.loads(val)
        coords = obj.get("coordinates", [])
        lon = coords[0] if len(coords) > 0 else None
        lat = coords[1] if len(coords) > 1 else None
        alt = coords[2] if len(coords) > 2 else None
        gtype = obj.get("type")
        return pd.Series([lat, lon, alt, gtype])
    except Exception:
        return pd.Series([None, None, None, None])

#Appliquer au dataframe
df_sites[["Latitude", "Longitude", "Altitude", "Geo_type"]] = (df_sites["Geo Shape"].astype(str).apply(parse_geo_shape))

display(df_sites.head(5))

,Geo Point,Geo Shape,dep,adresse,nomcom,insee,nom_carto,typo_niv3,objectid,Latitude,Longitude,Altitude,Geo_type
0,"48.8675639747, 2.78603920275","{""coordinates"": [2.786039202752757, 48.8675639...",77,77777 MARNE-LA-VALLÉE CEDEX 4,MARNE LA VALLÉE,77535,Business solutions - Disneyland® Paris,Centre de congrès et parc d'exposition,7,48.867564,2.786039,0.0,Point
1,"48.8229494567, 2.26979827721","{""coordinates"": [2.269798277205975, 48.8229494...",92,25 AVENUE VICTOR CRESSON 92130 ISSY LES MOULIN...,ISSY LES MOULINEAUX,92040,LE PALAIS DES CONGRÈS D’ISSY,Centre de congrès et parc d'exposition,12,48.822949,2.269798,0.0,Point
2,"48.8719414131, 2.21752817734","{""coordinates"": [2.217528177341401, 48.8719414...",92,123 BD WASHINGTON,SURESNES,92073,Cimetiere Americain,Site et monument,55,48.871941,2.217528,0.0,Point
3,"48.4265659583, 2.16097168091","{""coordinates"": [2.160971680905903, 48.4265659...",91,"5, AVENUE CHARLES-DE-GAULLE",ETAMPES,91223,Ile de loisirs d'Etampes,"Espace de loisirs, espace récréatif",393,48.426566,2.160972,0.0,Point
4,"49.0309724221, 2.04739741079","{""coordinates"": [2.047397410785327, 49.0309724...",95,RUE DES ETANGS,CERGY-PONTOISE,95127,Ile de loisirs de Cergy Pontoise,"Espace de loisirs, espace récréatif",404,49.030972,2.047397,0.0,Point
